## Exercice 2 - BL

In [62]:
#Importing relevant libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [63]:
# Read in the data from the Excel file
data = pd.read_excel('BL_data_me.xlsx', 'Sheet1')

# Extract the relevant columns and set the index
df_corr = data.iloc[:, :8].dropna()
df_corr.set_index(data.columns[0], inplace=True)
df_weq = data.iloc[:, [0, 11]].dropna()
df_weq.set_index(data.columns[0], inplace=True)
df_sig = data.iloc[:, [0, 10]].dropna()
df_sig.set_index(data.columns[0], inplace=True)

# Set the parameters
gamma = 2
tau = 0.03
Om_diag = np.array([0.025, 0.015, 0.04]) * tau

In [64]:
N = df_sig.shape[0]
vec1 = np.ones((N, 1))
weq = df_weq.to_numpy().reshape((N, 1))
sig = df_sig.to_numpy().reshape((N, 1))
corr = df_corr.to_numpy()

Sigma = (sig @ vec1.T) * corr * (vec1 @ sig.T)
Sigma = np.multiply(Sigma, np.ones((N, N)))

## a) risk aversion coeffecient = 3.5 -> mu_0

In [65]:
mu0 = np.dot(Sigma, weq) * 3.5
mu0 = np.around(mu0, 4)
mu0_df = pd.DataFrame({'mu_0': mu0.flatten()}, index=df_weq.index)

display(mu0_df)

,mu_0
Unnamed: 0,
Australia,0.0551
Canada,0.0968
France,0.1170
Germany,0.1264
Japan,0.0602
UK,0.0947
US,0.1058


## b)

In [66]:
mu0 = np.dot(Sigma, weq) * gamma
Omega0 = tau * Sigma
mu_hat = mu0
Sigma_hat = Sigma + Omega0

w0 = np.dot(np.linalg.inv(gamma * Sigma_hat), mu_hat)
w_diff = w0 - weq
w_df = pd.DataFrame({'w_0': w0.flatten(), 'w_0-w_eq': w_diff.flatten()}, index=df_weq.index)
w_df = np.around(w_df, 4)

w_df = w_df.drop(w_df.index[0], axis=0) if w_df.index[0] == 0 else w_df

display(w_df)






,w_0,w_0-w_eq
Unnamed: 0,,
Australia,0.0155,-0.0005
Canada,0.0214,-0.0006
France,0.0505,-0.0015
Germany,0.0534,-0.0016
Japan,0.1126,-0.0034
UK,0.1204,-0.0036
US,0.5971,-0.0179


## c)

In [88]:
q = np.array([0.045, 0.02]).reshape((2, 1))
P1 = np.array([0, 0, float(-weq[2] / (weq[2] + weq[5])), 1, 0, -float(weq[5] / (weq[2] + weq[5])), 0])
P2 = np.array([0, 1, 0, 0, 0, 0, -1])
P = np.vstack((P1, P2))

Omega = np.diag(Om_diag[:2])

# Posterior distribution
def posterior_dist(mu0, Omega0, q, Omega, P):
    A = P @ Omega0 @ P.T + Omega
    mu_hat = mu0 + Omega0 @ P.T @ np.linalg.solve(A, q - P @ mu0)
    Omega_hat = Omega0 - Omega0 @ P.T @ np.linalg.solve(A, P @ Omega0)
    return mu_hat, Omega_hat

# Formula 1:
mu_hat, Omega_hat = posterior_dist(mu0, Omega0, q, Omega, P)
print('The mean of post distribution:')
display(pd.DataFrame(data=np.around(mu_hat, 4), columns=['mu_hat'], index=df_weq.index))
print('and variance-covariance matrix is given by :')
display(pd.DataFrame(data=np.around(Omega_hat, 4), columns=df_weq.index, index=df_weq.index))

# Formula 2:
invOmega0 = np.linalg.inv(Omega0)
invOmega = np.linalg.inv(Omega)

A = P.T @ invOmega @ P + invOmega0
mu_hat2 = np.linalg.solve(A, invOmega0 @ mu0 + P.T @ invOmega @ q)
Omega_hat2 = np.linalg.inv(A)

# Verify that both formulas agree:
print('By comparing the formulas in the slides we get the same answer:')
print(np.max(np.abs(mu_hat - mu_hat2)))
print(np.max(np.abs(Omega_hat - Omega_hat2)))

The mean of post distribution:


,mu_hat
Unnamed: 0,
Australia,0.0356
Canada,0.0695
France,0.0765
Germany,0.0912
Japan,0.0370
UK,0.0559
US,0.0607


and variance-covariance matrix is given by :


Unnamed: 0,Australia,Canada,France,Germany,Japan,UK,US
Unnamed: 0,,,,,,,
Australia,0.0007,0.0004,0.0005,0.0006,0.0004,0.0005,0.0004
Canada,0.0004,0.0011,0.0009,0.0009,0.0004,0.0007,0.0009
France,0.0005,0.0009,0.0017,0.0015,0.0005,0.0011,0.0009
Germany,0.0006,0.0009,0.0015,0.0017,0.0006,0.0012,0.0009
Japan,0.0004,0.0004,0.0005,0.0006,0.0013,0.0005,0.0004
UK,0.0005,0.0007,0.0011,0.0012,0.0005,0.0012,0.0007
US,0.0004,0.0009,0.0009,0.0009,0.0004,0.0007,0.0010


By comparing the formulas in the slides we get the same answer:
2.0816681711721685e-17
4.336808689942018e-19


In [78]:
def black_litterman(mu0,q,Omega,tau,P,gamma):
    
    Lambda_1    = np.linalg.inv((P@Sigma)@P.T+(1+1/tau)*Omega)
    Lambda_2 =(q-1/(1+tau)*(P@mu0))/gamma
    Lambda = Lambda_1@Lambda_2
    Omega0     = tau*Sigma
    w          = P.T@Lambda + w0
    return w,Lambda

ws,Lambda  = black_litterman(mu0,q,Omega,tau,P,gamma)

display(pd.DataFrame(data=np.around(np.concatenate((ws,np.add(ws,-w0),np.add(ws,-weq)),axis=1),4),\
     columns=['w*','w*-w_0','w*-w_eq'],index=df_weq.index))

,w*,w*-w_0,w*-w_eq
Unnamed: 0,,,
Australia,0.0155,0.0000,-0.0005
Canada,0.3868,0.3655,0.3648
France,-0.0426,-0.0930,-0.0946
Germany,0.3683,0.3149,0.3133
Japan,0.1126,0.0000,-0.0034
UK,-0.1015,-0.2219,-0.2255
US,0.2316,-0.3655,-0.3834


In [83]:
# The weights -> lambdas

lambdas_df = pd.DataFrame(Lambda, columns=['View weights'], index=['Lambda 1', 'Lambda 2'])
display(lambdas_df)

,View weights
Lambda 1,0.314920
Lambda 2,0.365477


In [87]:
# Define inputs
q = np.array([0.045, 0.02, 0.055]).reshape((3, 1))
P3 = np.array([0, 0, 0, 0, 1, 0, 0])
P = np.vstack((P1, P2, P3))
Omega = np.diag(Om_diag)

# Calculate posterior distribution
mu_hat, Omega_hat = posterior_dist(mu0, Omega0, q, Omega, P)

# Calculate portfolio weights and lambda
ws, Lambda = black_litterman(mu0, q, Omega, tau, P, gamma)

# Display results
display(pd.DataFrame(ws, columns=['w*'], index=df_weq.index))
display(pd.DataFrame(Lambda, columns=['View-weights'], index=['lambda 1', 'lambda 2 ', 'lambda 3 ']))

,w*
Unnamed: 0,
Australia,0.015534
Canada,0.383102
France,-0.040714
Germany,0.362072
Japan,0.224265
UK,-0.097086
US,0.235344


,View-weights
lambda 1,0.308674
lambda 2,0.361743
lambda 3 $,0.111644
